In [7]:
!pip install DialogTag
!pip install StanfordCoreNLP
!pip install pycorenlp
!pip install dialog_tag
!pip install stemming
!pip install textract

ERROR: Could not find a version that satisfies the requirement dialog_tag (from versions: none)
ERROR: No matching distribution found for dialog_tag


In [3]:
# Import our python files
import categorize_relevance, find_questions, preprocessing
# Note that this might take some time to load

In [8]:
# For demonstration purposes, we will use one of our conversations from our dataset
transcription_file = "example_conversation.txt"
wiki_tfidf_file = "wiki_tfidf_terms.csv"

In [9]:
# Preprocess our text file into speakerturns
speakerturns = preprocessing.preprocessing(transcription_file)
# Turn our speakerturns into a dataframe, so we can label our data
df = preprocessing.create_labeled_dataframe(speakerturns)

In [10]:
# This is what our data currently looks like
df.head()

,identifier,time,speaker,text,question,relevant
identifier,,,,,,
0,0,[0:00:00],spk_0,we're doing I'm *** and this is **** we are co...,0,0
1,1,[0:01:17],spk_1,Expense category. So the budget consists of in...,0,0
2,2,[0:02:15],spk_2,"Okay, so the IFA is like an association of dif...",0,0
3,3,[0:02:25],spk_1,No no no no no no. Okay. So the idea is that e...,0,0
4,4,[0:02:43],spk_0,"Sorry, sorry to interrupt.",0,0


In [11]:
# Finding our questions, using Dialog Acts in our case
!pip install nltk
import nltk
nltk.download('punkt')
df_q = find_questions.dialog_act_questions(df)
# This might take a while...
# Alternatively one could use POS tags:
# df_q = find_questions.pos_questions(df) # Note that this requires StanfordCoreNLP to be ran on localhost

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


bert-base-uncased found in cache. Loading model...


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /root/.dialog-tag/models/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


1/1 [==============================] - 0s 60ms/step


In [12]:
# Now our data looks like this, when looking at some of the questions
df_q[df_q.question == 1].head()

,identifier,time,speaker,text,question,relevant
identifier,,,,,,
0,0,[0:00:00],spk_0,we're doing I'm *** and this is **** we are co...,1,0
1,1,[0:01:17],spk_1,Expense category. So the budget consists of in...,1,0
2,2,[0:02:15],spk_2,"Okay, so the IFA is like an association of dif...",1,0
12,12,[0:03:51],spk_0,Okay. All right. Um and um can you uh give us ...,1,0
13,13,[0:04:11],spk_1,for the budgeting system? I think the IFA admi...,1,0


In [84]:
from itertools import islice
from nltk import tokenize
from numpy import array, log
import re
from sklearn.feature_extraction.text import CountVectorizer
from stemming.porter2 import stem
import textract
from tqdm import tqdm
import pandas as pd
import numpy as np
!pip install stemming
!pip install textract
!pip install tqdm
!pip install pandas

In [1]:
# Now, we will create our relevance indicating words list from our original transcription file
# Here, we take 60 of the words after which we filter them, using our wikipedia tfidf file
from categorize_relevance import tfidf_word_list, filter_questions

# Specify the file paths
transcription_file = "example_conversation.txt"
wiki_tfidf_file = "wiki_tfidf_terms.csv"

# Now, we will create our relevance indicating words list from our original transcription file
# Here, we take 60 of the words after which we filter them, using our wikipedia tfidf file
relevance_words = tfidf_word_list(transcription_file, 60, wiki_tfidf_file)
# This will take a while, as we process a big TF-IDF file from the wikipedia dump

  1%|▏         | 130615/10394041 [00:01<01:27, 117165.77it/s]
130647it [00:01, 116497.00it/s]
 52%|█████▏    | 5366140/10394041 [00:19<00:20, 249227.23it/s]
5366226it [00:19, 242625.33it/s]
5391621it [00:20, 224771.50it/s]

Error on: "−7,3,13,23..",1,5989879,15.605581769553712



 52%|█████▏    | 5414971/10394041 [00:20<00:22, 217488.48it/s]
5414987it [00:20, 212313.57it/s]
 60%|█████▉    | 6230028/10394041 [00:24<00:13, 315155.25it/s]

Error on: "2,4,6,8..",1,5989879,15.605581769553712




 73%|███████▎  | 7576566/10394041 [00:28<00:08, 316211.32it/s]
7577536it [00:28, 315307.64it/s]

Error on: "১,৭৫,৫৭,৩৪৫",1,5989879,15.605581769553712



 79%|███████▉  | 8257581/10394041 [00:30<00:06, 327915.31it/s]
8259404it [00:30, 328876.55it/s]

Error on: ",3f.d.m.,3s.h.p.d.,8s.h.p.d",1,5989879,15.605581769553712



100%|█████████▉| 10381856/10394041 [00:39<00:00, 293437.02it/s]
10394040it [00:39, 265428.90it/s]
100%|██████████| 10099022/10099022 [02:06<00:00, 80146.05it/s]


Keywords of transcript according to wiki TF-IDF ['acces', 'paraphrasing', 'encrypted', 'budgetting', 'flustered', 'avoid', 'assignment', 'sequence', 'install', 'improvements', 'initiator', 'storage', 'reasoning', 've', 'eliminate', 'separately', 'push', 'please', 'ifa', 'tasked', 'administer', 'enhance', 'demarcate', 'expenses', 'suppose', 'leakage', 'insert', 'sites', 'portal', 'activate', 'owner', 'nowadays', 'scenario', 'hear', 'wording', 'admins', 'manually', 'app', 'external', 'scores', 'request', 'executing', 'talking', 'gambling', 'define', 'interrupt', 'concerns', 'technical']


In [13]:
# Finally, we filter out our questions using these relevance indicating words
df_rq = categorize_relevance.filter_questions(df_q, relevance_words)

In [14]:
# Our relevant questions finally look like this:
df_rq[df_rq.relevant == 1].head()
output_filename = 'output_dataframe.xlsx'
df_q.to_excel(output_filename, index=False)  # index=False to exclude the index column

print(f"DataFrame exported to {output_filename}")

DataFrame exported to output_dataframe.xlsx
